In [1]:
import os

In [2]:
%pwd

'd:\\Test Summ\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Test Summ'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path 

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 param_filepath =PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.param = read_yaml(param_filepath)
        
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )
        
        return data_transformation_config     

In [8]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

[2024-01-14 20:48:33,337: INFO: config: PyTorch version 2.1.1 available.]
[2024-01-14 20:48:33,362: INFO: config: TensorFlow version 2.15.0 available.]


In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer =  AutoTokenizer.from_pretrained(config.tokenizer_name) 
        
    
    def convert_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'],max_length=1024,truncation=True) 
        
        with self.tokenizer.as_target_tokenizer():
            traget_encoding = self.tokenizer(example_batch['summary'],max_length=128,truncation=True)  
            
            return {
                'input_ids' : input_encodings['input_ids'],
                'attention_mask' : input_encodings['attention_mask'],
                'labels': traget_encoding['input_ids']
            }  
            
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)  
        dataset_samsum_pt = dataset_samsum.map(self.convert_to_features,batched = True)      
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset")) 
        
        
            
            

In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e 

[2024-01-14 20:48:39,978: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-14 20:48:39,980: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-14 20:48:39,983: INFO: common: File Created artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

C:\Users\Adil\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]